# 📚 **Installation**

In [1]:
import bz2
import pandas as pd
import joblib
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module='category_encoders.base_contrast_encoder')
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)


# 📂 **Data**

In [2]:
result_df = pd.read_csv("FE_golf.csv")

# 🤖 **Model Import**

In [3]:
#load saved compressed approach model
with bz2.BZ2File('approachModel.joblib.bz2', 'rb') as f:
    approachModel = joblib.load(f)

#load saved compressed putting model
with bz2.BZ2File('puttingModel.joblib.bz2', 'rb') as f:
    puttingModel = joblib.load(f)

# **⛳ Expected Strokes Predction**

In [4]:
result_df['xS'] = 0

#approach
bunker_test = result_df.iloc[17]
f = bunker_test
input = f.to_frame().T

#green
putting_test = result_df.iloc[2]
f = putting_test
putting_input = f.to_frame().T

for index, row in result_df.iterrows():

    if row['from_location_scorer'] == 'Green':

        putting_input['slope'] = row['slope']
        putting_input['elevation'] = row['elevation']
        putting_input['round'] = row['round']
        putting_input['par_value'] = row['par_value']
        putting_input['putting_dist_from_center_bins'] = row['putting_dist_from_center_bins']
        putting_input['putting_dist_from_edge_bins'] = row['putting_dist_from_edge_bins']
        putting_input['putting_distance_to_pin_bins'] = row['putting_distance_to_pin_bins']
        putting_input['first_putt'] = row['first_putt']

        putting_input['ISS'] = row['ISS']
        putting_input['ISC'] = row['ISC']
        putting_input['SSI'] = row['SSI']
        putting_input['distance_to_pin'] = row['distance_to_pin']
        putting_input['dist_from_edge'] = row['dist_from_edge']
        putting_input['pin_minus_edge'] = row['pin_minus_edge']
        putting_input['hole_completeness'] = row['hole_completeness']
        putting_input['dist_from_center'] = row['dist_from_center']

        preds = puttingModel.predict(putting_input)
        result_df['xS'].iloc[index] = preds[0]

    if row['from_location_scorer'] != 'Green':

        input['round'] = row['round']
        input['par_value'] = row['par_value']
        input['lie'] = row['lie']
        input['slope'] = row['slope']
        input['elevation'] = row['elevation']
        input['non_putting_dist_from_center_bins'] = row['non_putting_dist_from_center_bins']
        input['non_putting_dist_from_edge_bins'] = row['non_putting_dist_from_edge_bins']
        input['non_putting_distance_to_pin_bins'] = row['non_putting_distance_to_pin_bins']
        input['around_the_green'] = row['around_the_green']
        input['side_of_hole'] = row['side_of_hole']
        input['dog_legs'] = row['dog_legs']
        input['HCP'] = row['HCP']
        input['shot_type'] = row['shot_type']
        input['from_location_scorer'] = row['from_location_scorer']
        input['from_location_laser'] = row['from_location_laser']

        input['ISS'] = row['ISS']
        input['ISC'] = row['ISC'] 
        input['SSI'] = row['SSI']
        input['CSI'] = row['CSI']
        input['shot'] = row['shot']
        input['distance_to_pin'] = row['distance_to_pin']
        input['dist_from_edge'] = row['dist_from_edge']
        input['dist_from_center'] = row['dist_from_center']
        input['pin_minus_edge'] = row['pin_minus_edge']
        input['Actual Yard'] = row['Actual Yard']
        input['yardage'] = row['yardage']
        input['hole_completeness'] = row['hole_completeness']


        preds = approachModel.predict(input)
        result_df['xS'].iloc[index] = preds[0]

C:\Users\decla\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\decla\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


# **🏌🏻 Strokes Gained Creation**

In [6]:
result_df['SG'] = result_df['xS'] - result_df['strokes_to_hole_out']

# 📩 **Save**

In [7]:
result_df.to_csv('SG.csv')